<h1 align="center"><font size="5">Week 3 Assignment</font></h1>

In this notebook, we will explore, segment, and cluster the neighborhoods in the city of Toronto. The neighborhood data is not readily available on the internet.

## Part 1: Build a dataframe of the postal code

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


+ Obtain the data in the table of postal codes.


+ Transform the data into a *pandas* dataframe.

**Requirements**

1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.


3. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.


   **at the time I do this assignment, the table on the Wikipedia page lists all neigborhoods into ONE postal code.**

4. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


5. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.


6. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.



In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
# scrape the wikipedia page to obtain the data in the table of postal codes
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

source=requests.get(url).text

soup = BeautifulSoup(source,'xml')

table=soup.find('table') # find a table from url

In [3]:
# transform the data into pandas dataframe
# dataframe consists of 3 columns: Postalcode, Borough, Neighborhood
column_names=['Postal Code','Borough','Neighborhood']
df=pd.DataFrame(columns=column_names) 

In [4]:
# Search all the Postalcode, Borough, Neighborhood

for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)]=row_data 

df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
# remove rows where Borough is Not assigned
df = df[df['Borough']!='Not assigned']

In [6]:
# if a cell has a Borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

In [7]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


## Part 2: Build a dataframe with latitude and longitude coordinates of each neighborhood

In [9]:
!pip install geocoder

In [10]:
import geocoder # import geocoder

def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [11]:
geo_df=pd.read_csv('http://cocl.us/Geospatial_data')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
# merge 2 dataframe df and geo_df
geo_merged = pd.merge(geo_df, df)

In [14]:
data=geo_merged[['Postal Code', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [15]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(data['Borough'].unique()), data.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


## Part 3: Explore and Cluster the neighborhoods in Toronto

Replicate the same analysis we did to the NYC data in the lab cognitive class.

## 3.1 Explore Dataset

Work with only borough that contain the word 'Central Toronto.'

In [16]:
!pip install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --yes


### Use geopy library to get the latitude and longitude values of Toronto

In [17]:
# in order to define an instance of the geocode, we need to define a user_agent. 
# Name our agent toronto_explorer
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Creat a map of Toronto with neighborhoods superimposed on top

In [18]:
!pip install folium
import folium # map rendering library

In [19]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [20]:
print('Total number of Borough =', len(df['Borough'].unique()))

Total number of Borough = 10


For illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in West Toronto. So let's **slice the original dataframe and create a new dataframe of the Central Toronto.**

In [21]:
central_toronto=data[data['Borough']=="Central Toronto"].reset_index(drop=True)
central_toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
5,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
6,M5N,Central Toronto,Roselawn,43.711695,-79.416936
7,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
8,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678


Let's get the gepgraphical coordinates of Central Toronto.

### Use geopy library to get the latitude and longitude values of Central Toronto

In [22]:
address = 'Central Toronto, Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


### Creat a map of Central Toronto with neighborhoods superimposed on top

In [38]:
# visualize East Toronto the neighborhoods in it.
# create map of East Toronto using latitude and longitude values
map_central_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(central_toronto['Latitude'], central_toronto['Longitude'], central_toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central_toronto)  
    
map_central_toronto

We start ultilizing the Foursquare API to explore the neighborhoods and segment them.

### Define Foursquare Credentials and Version

In [23]:
# hidden cell
CLIENT_ID = 'AYDKHDCJEGJVBUHJC0BA5VPYY05XSCJQVJGBMLJPJTQFK0FV' # your Foursquare ID
CLIENT_SECRET = '3TO3YGTN3EGFLJ3ADIGUQZIC0VOMDAIZLAYJ34NEVVNIBJUS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AYDKHDCJEGJVBUHJC0BA5VPYY05XSCJQVJGBMLJPJTQFK0FV
CLIENT_SECRET:3TO3YGTN3EGFLJ3ADIGUQZIC0VOMDAIZLAYJ34NEVVNIBJUS


### Explore the first neighborhood in our dataframe

In [24]:
# Get the neighborhood's name
central_toronto.loc[0, 'Neighborhood']

'Lawrence Park'

In [25]:
# Get the neighborhood's latitude and longitude values
neighborhood_latitude = central_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = central_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = central_toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


### Get the venues that are in Lawrence Park within a radius of 500 meters.

In [26]:
# Create the GET request URL, name your URL as url_explore
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url_explore = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url_explore# display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AYDKHDCJEGJVBUHJC0BA5VPYY05XSCJQVJGBMLJPJTQFK0FV&client_secret=3TO3YGTN3EGFLJ3ADIGUQZIC0VOMDAIZLAYJ34NEVVNIBJUS&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=100'

In [27]:
# send the GET request and examine the results
results = requests.get(url_explore).json()
results

{'meta': {'code': 200, 'requestId': '5ebcaf19618f43001b830654'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.7325205045, 'lng': -79.3825744605273},
   'sw': {'lat': 43.7235204955, 'lng': -79.3950057394727}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledL

All the information is in the *items* key. 

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [29]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [30]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [31]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


## 3.2 Explore Neighborhoods in Central Toronto

In [32]:
# create a function to repeat the same process to all the neighborhoods in Central Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [33]:
# write the code to runn the above function on each neighborhood
# and create a new dataframe called central_toronto_venues
central_toronto_venues = getNearbyVenues(names=central_toronto['Neighborhood'],
                                   latitudes=central_toronto['Latitude'],
                                   longitudes=central_toronto['Longitude']
                                  )

Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville


In [34]:
# check the size of the resulting dataframe
print(central_toronto_venues.shape)
central_toronto_venues.head()

(115, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,Homeway Restaurant & Brunch,43.712641,-79.391557,Breakfast Spot
4,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park


In [35]:
# check how many venues were returned for each neighborhood
central_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,35,35,35,35,35,35
Davisville North,7,7,7,7,7,7
Forest Hill North & West,4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",2,2,2,2,2,2
North Toronto West,22,22,22,22,22,22
Roselawn,1,1,1,1,1,1
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",17,17,17,17,17,17
"The Annex, North Midtown, Yorkville",24,24,24,24,24,24


In [36]:
# how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(central_toronto_venues['Venue Category'].unique())))

There are 65 uniques categories.


## 3.3 Analyze each neighborhood

In [37]:
# one hot encoding
central_toronto_onehot = pd.get_dummies(central_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_toronto_onehot['Neighborhood'] = central_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [central_toronto_onehot.columns[-1]] + list(central_toronto_onehot.columns[:-1])
central_toronto_onehot = central_toronto_onehot[fixed_columns]

central_toronto_onehot.head()

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,...,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Davisville North,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
central_toronto_onehot.shape

(115, 66)

In [39]:
# group rows by neighborhood and by taking the mean of the frequency of occurence of each category
central_toronto_grouped=central_toronto_onehot.groupby('Neighborhood').mean().reset_index()
central_toronto_grouped

,Neighborhood,American Restaurant,BBQ Joint,Bagel Shop,Bank,Bar,Breakfast Spot,Brewery,Burger Joint,Bus Line,...,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.028571,0.000000,0.000000,...,0.000000,0.000000,0.057143,0.000000,0.028571,0.028571,0.00,0.000000,0.000000,0.000000
1,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
2,Forest Hill North & West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000
3,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,...,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
4,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
5,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.045455
6,Roselawn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
7,"Summerhill West, Rathnelly, South Hill, Forest...",0.058824,0.000000,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.058824,0.058824,0.058824,0.000000,0.000000,0.000000,0.00,0.000000,0.058824,0.000000
8,"The Annex, North Midtown, Yorkville",0.041667,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.041667,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.041667,0.000000,0.000000


In [40]:
# confirm the new size
central_toronto_grouped.shape

(9, 66)

In [41]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in central_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = central_toronto_grouped[central_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0      Sandwich Place  0.09
1        Dessert Shop  0.09
2         Pizza Place  0.06
3  Italian Restaurant  0.06
4                 Gym  0.06


----Davisville North----
               venue  freq
0   Department Store  0.14
1     Sandwich Place  0.14
2                Gym  0.14
3               Park  0.14
4  Food & Drink Shop  0.14


----Forest Hill North & West----
                venue  freq
0    Sushi Restaurant  0.25
1  Mexican Restaurant  0.25
2               Trail  0.25
3       Jewelry Store  0.25
4          Playground  0.00


----Lawrence Park----
          venue  freq
0      Bus Line  0.33
1          Park  0.33
2   Swim School  0.33
3  Liquor Store  0.00
4    Playground  0.00


----Moore Park, Summerhill East----
                 venue  freq
0                  Gym   0.5
1           Playground   0.5
2  American Restaurant   0.0
3  Rental Car Location   0.0
4    Indian Restaurant   0.0


----North Toronto West----
                     ven

### Put that into a *pandas* dataframe

In [42]:
# define a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [43]:
import numpy as np

In [44]:
# create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
df_venues_sorted = pd.DataFrame(columns=columns)
df_venues_sorted['Neighborhood'] = central_toronto_grouped['Neighborhood']

for ind in np.arange(central_toronto_grouped.shape[0]):
    df_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_toronto_grouped.iloc[ind, :], num_top_venues)

df_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Sandwich Place,Dessert Shop,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Gym,Café,Restaurant,Dance Studio
1,Davisville North,Food & Drink Shop,Breakfast Spot,Park,Sandwich Place,Department Store,Gym,Hotel,Gas Station,Garden,Fried Chicken Joint
2,Forest Hill North & West,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Department Store,Dessert Shop,Diner,Donut Shop,Farmers Market
3,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Fast Food Restaurant,Dessert Shop,Diner,Donut Shop,Farmers Market,Food & Drink Shop
4,"Moore Park, Summerhill East",Gym,Playground,Deli / Bodega,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint,Food & Drink Shop


## 3.4 Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [45]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [46]:
# set number of clusters
kclusters = 5

central_toronto_grouped_clustering = central_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(central_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 3, 4, 2, 0, 1, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [47]:
# add clustering labels
df_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

central_toronto_merged = central_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
central_toronto_merged = central_toronto_merged.join(df_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

central_toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Fast Food Restaurant,Dessert Shop,Diner,Donut Shop,Farmers Market,Food & Drink Shop
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Food & Drink Shop,Breakfast Spot,Park,Sandwich Place,Department Store,Gym,Hotel,Gas Station,Garden,Fried Chicken Joint
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Yoga Studio,Restaurant,Café,Chinese Restaurant,Cosmetics Shop,Dessert Shop,Diner,Fast Food Restaurant
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Gym,Café,Restaurant,Dance Studio
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2,Gym,Playground,Deli / Bodega,Grocery Store,Greek Restaurant,Gourmet Shop,Gas Station,Garden,Fried Chicken Joint,Food & Drink Shop


Visualize the resulting clusters.

In [48]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [49]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(central_toronto_merged['Latitude'], central_toronto_merged['Longitude'], central_toronto_merged['Neighborhood'], central_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
# Examine clusters
# Cluster 1
central_toronto_merged.loc[central_toronto_merged['Cluster Labels'] == 0, central_toronto_merged.columns[[1] + list(range(5, central_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Central Toronto,0,Food & Drink Shop,Breakfast Spot,Park,Sandwich Place,Department Store,Gym,Hotel,Gas Station,Garden,Fried Chicken Joint
2,Central Toronto,0,Clothing Store,Coffee Shop,Yoga Studio,Restaurant,Café,Chinese Restaurant,Cosmetics Shop,Dessert Shop,Diner,Fast Food Restaurant
3,Central Toronto,0,Sandwich Place,Dessert Shop,Italian Restaurant,Coffee Shop,Sushi Restaurant,Pizza Place,Gym,Café,Restaurant,Dance Studio
5,Central Toronto,0,Coffee Shop,Pub,Health & Beauty Service,Restaurant,Bagel Shop,Bank,Fried Chicken Joint,Vietnamese Restaurant,Light Rail Station,Liquor Store
8,Central Toronto,0,Café,Sandwich Place,Coffee Shop,American Restaurant,Grocery Store,History Museum,Donut Shop,Indian Restaurant,Liquor Store,Middle Eastern Restaurant
